In [1]:
%reload_ext autoreload
%autoreload 2

import argparse
import os
import glob
import numpy as np
import torch
from networks.mmt import MMT as generator
from torchvision.utils import save_image
from tqdm import tqdm
from configs.config import get_config
from datasets.dataset_brats import BRATS_dataset, RandomGeneratorBRATS
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import Upsample

split = 'test'
batch_size = 1
data_dir = '/mnt/raid/jiang/projects/SubtleGAN/data/brats2021_slices_crop192x160'
ckpt_path_zerogad = '/mnt/raid/jiang/projects/SubtleGAN/MMTUNetHybrid/model/fluidstack_models/finetuned_models/MMT_single_epo100_bs32_lrg0.0005_5.0_20.0_0.0_0.1/best_mae.pth'
ckpt_path_random = '/mnt/raid/jiang/projects/SubtleGAN/model_fluidstack/MMT/MMTUNetHybrid_GAN_ls_random_s_c_MMT_epo75_bs32_lrg0.0005_192_opt-adamw/epoch_74.pth'
ckpt_path_single = '/mnt/raid/jiang/projects/SubtleGAN/model/MMT/MMTUNetHybrid_GAN_s_c_MMT_epo50_bs24_lrg0.0005_192_opt-adamw/epoch_49.pth'

ckpt_path = ckpt_path_zerogad

db = BRATS_dataset(base_dir=data_dir, split=split,
                               transform=transforms.Compose(
                                   [RandomGeneratorBRATS(flip=False, scale=None)]))
dataloader = DataLoader(db, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
G = generator(img_size=[160, 192],
                                patch_size=4,
                                in_chans=1,
                                out_chans = 1,
                                embed_dim=96,
                                depths=[2,2,2,2],
                                num_heads=[3,6,12,24],
                                window_size=[5,6],
                                mlp_ratio=4.0,
                                qkv_bias=True,
                                qk_scale=None,
                                drop_rate=0,
                                drop_path_rate=0,
                                ape=False,
                                patch_norm=True,
                                use_checkpoint=False).cuda()
state_dict = torch.load(ckpt_path, map_location='cpu')
G.load_state_dict(state_dict['G'])
G.eval()


SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0


/home/srivathsa/miniconda3/envs/mmt/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


---final upsample expand_first---


MMT(
  (heads): ModuleList(
    (0): Head(
      (model): Sequential(
        (0): Conv2d(1, 6, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (1): ResBlock(
          (body): Sequential(
            (0): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
            (1): ReLU(inplace=True)
            (2): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          )
        )
        (2): ResBlock(
          (body): Sequential(
            (0): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
            (1): ReLU(inplace=True)
            (2): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          )
        )
      )
    )
    (1): Head(
      (model): Sequential(
        (0): Conv2d(1, 6, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (1): ResBlock(
          (body): Sequential(
            (0): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
            (1): ReLU(inplace=True)
    

In [2]:
def slice_loader(fn):
    data = np.load(fn)
    n_channel = data.shape[0]
    image = torch.from_numpy(data.astype(np.float32)).unsqueeze(0)
    output = [image[:, i, :, :].unsqueeze(0).detach() for i in range(n_channel)]
    return output

In [3]:
def split_data(data, inputs, targets):
    contrast_input = inputs
    contrast_output = targets
    img_inputs = [data[i].detach().cuda() for i in contrast_input]
    img_targets = [data[i].detach().cuda() for i in contrast_output]
    return img_inputs, img_targets, contrast_input, contrast_output

In [4]:
from torch.nn.functional import interpolate
import torch
from torch.nn.functional import softmax

def sum_attn(attn_maps, head_normalization=True):
    n_heads = [24, 24, 12, 12, 6, 6, 3, 3]
    attn = torch.zeros_like(attn_maps[-1][:, :, :, :, 0])
    for n_head, attn_map in zip(n_heads, attn_maps):
        attn_map = attn_map[:, :, :, :, 0]
        scale_factor = 48//attn_map.shape[-1]
        attn_map = interpolate(attn_map, size=(40, 48))/scale_factor**2
        if head_normalization:
            attn_map = attn_map/n_head
        attn += attn_map
    return attn

In [5]:
data = slice_loader('/mnt/raid/jiang/projects/SubtleGAN/data/brats2021_slices_crop192x160/test/BraTS2021_01409/077.npy')
data = [d.detach().cuda() for d in data]
img_inputs, img_targets, contrast_input, contrast_output = split_data(data, [0,2,3], [1])

In [6]:
outputs, _, attn = G(img_inputs, contrast_input, contrast_output, return_attention=True)

cnn output torch.Size([1, 6, 160, 192])


In [ ]:
att_sum = sum_attn(attn)
att_sum = softmax(att_sum, dim = 1)
att_sum = torch.rot90(att_sum, 3, [2, 3])

In [ ]:
att_sum = att_sum.detach().cpu().numpy()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(att_sum[0, 0, :, :])
plt.colorbar()
plt.axis('off')
#plt.clim(0, 1)
plt.savefig("Fig/attn_t1.png")

In [ ]:
plt.imshow(att_sum[0, 1, :, :])
plt.colorbar()
plt.axis('off')
plt.savefig("Fig/attn_t2.png")

In [ ]:
plt.imshow(att_sum[0, 2, :, :])
plt.colorbar()
plt.axis('off')
plt.savefig("Fig/attn_flair.png")

In [ ]:
att_last = torch.rot90(attn[-1], 3, [2, 3])
att_last = softmax(att_last[:, :, :, :, 0], dim=1)
att_last = att_last.detach().cpu().numpy()

In [ ]:
plt.imshow(att_last[0, 0, :, :])
plt.colorbar()
plt.axis('off')
plt.savefig("Fig/attn_t1_last.png")

In [ ]:
plt.imshow(att_last[0, 1, :, :])
plt.colorbar()
plt.axis('off')
plt.savefig("Fig/attn_t2_last.png")

In [ ]:
plt.imshow(att_last[0, 2, :, :])
plt.colorbar()
plt.axis('off')
plt.savefig("Fig/attn_flair_last.png")